# Text comparison with Anthropic Claude


# Setup

* [Bedrock boto3 setup notebook](https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/00_Intro/bedrock_boto3_setup.ipynb) notebook.


In [29]:
from io import StringIO
import json
import os
import sys

import boto3
import textwrap

In [8]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
"""General helper utilities the workshop notebooks"""
def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

## AWS Bedrock service confitguration

In [2]:
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"
os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrock-runtime.us-west-2.amazonaws.com"

In [3]:
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2',
    endpoint_url=f'https://bedrock-runtime.us-west-2.amazonaws.com'
)

---
# Invoking Bedrock with Boto3


## Request Syntax of InvokeModel

Example of `InvokeModel` API request for sending text to Anthropic Claude. Inference parameters in `textGenerationConfig` depends on the model 

### Inference paramerters of Anthropic Claude

- **temperature** tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- **top_p** less than one keeps only the smallest set of most probable tokens with probabilities that add up to top_p or higher for generation.
- **top_k** can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- **max_tokens_to_sample** is maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- **stop_sequences** are sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

```python
response = bedrock.invoke_model(
    body={"prompt":"this is where you place your input text",
        "max_tokens_to_sample":4096,
        "temperature":0.5,
        "top_k":250,
        "top_p":0.5,
        "stop_sequences":[]
    },
    modelId="anthropic.claude-v2", 
    accept=accept, 
    contentType=contentType
)

```

## Prompt

In [26]:
prompt = """

Human: Give the confidence score between 0.0 and 1.0 how similar text1 and text2. 
The response is in JSON ONLY as {"score": score}.

<text1>
The patient is an 81-year-old female, who is approximately 10 years status post total knee replacement performed in another state. She presented a couple of days ago to the office with worsening pain without injury and whose symptoms have been present for approximately a month following a possible urinary tract infection. The patient' knee was aspirated in the office and cultures were positive for Escherichia coli. The knee was copiously irrigated with 9 L of irrigant. A partial synovectomy was performed in all compartments. Minimal amount of polyethylene wear was noted. The total knee components were identified arthroscopically for future revision surgery.  The patient was awakened from
</text1>
<text2>
PREOPERATIVE DIAGNOSIS: , Septic left total knee arthroplasty.,POSTOPERATIVE DIAGNOSIS: , Septic left total knee arthroplasty.,OPERATION PERFORMED: , Arthroscopic irrigation and debridement of same with partial synovectomy.,ANESTHESIA:,  LMA.,ESTIMATED BLOOD LOSS:,  Minimal.,COMPLICATIONS: , None.,DRAINS:,  None.,INDICATIONS:,  The patient is an 81-year-old female, who is approximately 10 years status post total knee replacement performed in another state, who presented a couple of days ago to the office with worsening pain without injury and whose symptoms have been present for approximately a month following a possible urinary tract infection.  The patient' knee was aspirated in the office and cultures were positive for Escherichia coli.  She presents for operative therapy.,DESCRIPTION OF OPERATION: , After obtaining informed consent and the administration of antibiotics since her cultures had already been obtained, the patient was taken to the operating room and following satisfactory induction and the patient was placed on the table in supine position.  The left upper extremity was prepped and draped without a tourniquet.  The knee was injected with 30 mL of normal saline and standard arthroscopy portals were created.  The arthroscopy was inserted and a complete diagnostic was performed.  Arthroscopic pictures were taken throughout the procedure.  The knee was copiously irrigated with 9 L of irrigant.  A partial synovectomy was performed in all compartments.  Minimal amount of polyethylene wear was noted.  The total knee components were identified arthroscopically for future revision surgery.  The knee was then drained and the arthroscopic instruments were removed.  The portals were closed with 4-0 nylon and local anesthetic was injected.  A sterile dressing was applied and the patient was placed in a knee immobilizer, awakened from anesthesia and transported to the recovery room in stable condition and tolerated the procedure well.
</text2>


Assistant:"""

## Request body with prompt and inference parameters 

In [30]:
body = json.dumps({
    "prompt": prompt,
    "max_tokens_to_sample":4096,
    "temperature":0.9,
    "top_k":250,
    "top_p":0.5,
    "stop_sequences":[]
}) 

## Invoke foundation model 
Here sends the API request to Amazon Bedrock with specifying request parameters `modelId`, `accept`, and `contentType`. Following the prompt, the foundation model in Amazon Bedrock summarizes the text.

In [31]:
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get('body').read())
print_ww(response_body.get('completion'))

 {"score": 0.95}


## Invoke foundation model with steaming
Bedrock supports `invoke_model_with_response_stream` API providing `ResponseStream` that streams the output in form of chunks to stream the output such that the user could start consuming it as it is being generated by the model.

In [32]:
response = boto3_bedrock.invoke_model_with_response_stream(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"completion":" {\\"","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":"score","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":"\\":","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" 0","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":".","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":"9","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":"}","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":"","stop_reason":"stop_sequence","stop":"\\n\\nHuman:","amazon-bedrock-invocationMetrics":{"inputTokenCount":623,"outputTokenCount":11,"invocationLatency":6201,"firstByteLatency":4989}}'}}]

Instead of generating the entire output, Bedrock sends smaller chunks from the model. This can be displayed in a consumable manner as well.

In [33]:
from IPython.display import display_markdown,Markdown,clear_output

In [34]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

 {"score": 0.94}